In [1]:
import numpy as np
import pandas as pd
import time

np.random.seed(2)  # reproducible

In [2]:
N_STATES = 6   # the length of the 1 dimensional world
ACTIONS = ['left', 'right']     # available actions

EPSILON = 0.9   # greedy policy
ALPHA = 0.1     # learning rate
GAMMA = 0.9    # discount factor

MAX_EPISODES = 13   # maximum episodes

FRESH_TIME = 0.3    # refresh/pause time for one move used in time.sleep

In [3]:
def build_q_table(n_states, actions):
    table = pd.DataFrame(
        np.zeros((n_states, len(actions))),     # q_table initial values
        columns=actions,    # actions's name
    )
    #print(table)    # show table
    return table

In [4]:
def choose_action(state, q_table):
    # This is how to choose an action
    state_actions = q_table.iloc[state, :]

    if (np.random.uniform() > EPSILON) or ((state_actions == 0).all()):  # act non-greedy or state-action have no value
        action_name = np.random.choice(ACTIONS) #Exploration
    else:   # act greedy
        action_name = state_actions.idxmax()    # replace argmax to idxmax as argmax means a different function in newer version of pandas

    return action_name

In [5]:
#similar to env.step() ---> accepting state and action and returning reward and next state
def get_env_feedback(S, A):
    # This is how agent will interact with the environment
    if A == 'right':    # move right
        if S == N_STATES - 2:   # terminate
            S_ = 'terminal'
            R = 1
        else:
            S_ = S + 1
            R = 0
    else:   # move left
        R = 0
        if S == 0:
            S_ = S  # reach the wall
        else:
            S_ = S - 1
    return S_, R

In [6]:
def update_env(S, episode, step_counter):#env.render()
    # This is how environment be updated
    env_list = ['-']*(N_STATES-1) + ['T']   # '---------T' our environment
    if S == 'terminal':
        interaction = 'Episode %s: total_steps = %s' % (episode+1, step_counter)
        print('\r{}'.format(interaction), end='')
        time.sleep(2)
        print('\r                                ', end='')
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(FRESH_TIME)

In [7]:
def rl():
    # main part of RL loop
    q_table = build_q_table(N_STATES, ACTIONS)
    for episode in range(MAX_EPISODES):
        step_counter = 0
        S = 0
        is_terminated = False
        update_env(S, episode, step_counter)
        while not is_terminated:

            A = choose_action(S, q_table)
            S_, R = get_env_feedback(S, A)  # take action & get next state and reward
            q_predict = q_table.loc[S, A]
            if S_ != 'terminal':
                q_target = R + GAMMA * q_table.iloc[S_, :].max()   # next state is not terminal
            else:
                q_target = R     # next state is terminal
                is_terminated = True    # terminate this episode

            q_table.loc[S, A] += ALPHA * (q_target - q_predict)  # update
            S = S_  # move to next state

            update_env(S, episode, step_counter+1)
            step_counter += 1
    return q_table

In [8]:
q_table = rl()
print('\r\nQ-table:\n')
print(q_table)

                                
Q-table:

       left     right
0  0.000000  0.004320
1  0.000000  0.025005
2  0.000030  0.111241
3  0.000000  0.368750
4  0.027621  0.745813
5  0.000000  0.000000


In [9]:
#RL Deploy Code
state = 0 
iterations_no = 100 

# Ensuring the state is not terminal before taking action
while state != 'terminal' and iterations_no > 0:
    # Choose the best action using the Q-table
    action = q_table.iloc[state, :].idxmax()
    
    # Get new state and reward
    next_state, reward = get_env_feedback(state, action) 

    print("Next state: ", next_state)
    print("Reward: ", reward)
    print( "Iteration {} and action {}".format(iterations_no, action)) 

    # Update state and decrement the number of iterations
    state = next_state
    iterations_no -= 1
    
    # Pause to see the process
    time.sleep(2)  


Next state:  1
Reward:  0
Iteration 100 and action right
Next state:  2
Reward:  0
Iteration 99 and action right
Next state:  3
Reward:  0
Iteration 98 and action right
Next state:  4
Reward:  0
Iteration 97 and action right
Next state:  terminal
Reward:  1
Iteration 96 and action right


In [10]:
#RL Deploy Code
#state = 0 
#iterations_no = 100 
#for i in range(iterations_no):     
    #randomAction = ACTIONS[state]     
    #retnVal = get_env_feedback(state, randomAction)     
    #print("return Value ", retnVal)     
    #print( "iteration {} and action {}".format(i+1,                                       
      #randomAction))     
    #time.sleep(2)     
    #state = retnVal[0]     
    #if retnVal == state:         
        #break;

